In [11]:
#live youtube session by Siraj Naval
#https://github.com/llSourcell/wiki_generator_live/blob/master/democodewiki.ipynb
import numpy as np #vectorization
import random # generate text
import tensorflow as tf #ML
import datetime # clock training time

In [2]:
#dataset from https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
text = open('wikitext-103-raw/wiki.test.raw').read()
print('text length in number of characters {0}'.format(len(text)))

print('head of text:\n{0}'.format(text[:100]))

text length in number of characters 1288556
head of text:
 
 = Robert Boulter = 
 
 Robert Boulter is an English film , television and theatre actor . He had 


In [4]:
#print out characters and sort them
chars = sorted(list(set(text)))
char_size = len(chars)
print('number of characters', char_size)
print(chars)

number of characters 259
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', '¥', '©', '°', '½', 'Á', 'Æ', 'É', '×', 'ß', 'à', 'á', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'î', 'ñ', 'ó', 'ô', 'ö', 'ú', 'ü', 'ć', 'č', 'ě', 'ī', 'ł', 'Ō', 'ō', 'Š', 'ū', 'ž', 'ǐ', 'ǔ', 'ǜ', 'ə', 'ɛ', 'ɪ', 'ʊ', 'ˈ', 'ː', '̍', '͘', 'Π', 'Ω', 'έ', 'α', 'β', 'δ', 'ε', 'ι', 'λ', 'μ', 'ν', 'ο', 'π', 'ς', 'σ', 'τ', 'υ', 'ω', 'ό', 'П', 'в', 'д', 'и', 'к', 'н', 'א', 'ב', 'י', 'ל', 'ר', 'ש', 'ת', 'ا', 'ت', 'د', 'س', 'ك', 'ل', 'و', 'ڠ', 'ग', 'न', 'र', 'ल', 'ष', 'ु', 'े', 'ो', '्', 'ả', 'ẩ', '‑', '–', '—', '’', '“'

In [8]:
char2id = dict((c, i) for i, c in enumerate(chars))
id2char = dict((i, c) for i, c in enumerate(chars))

In [9]:
#generate probability of each character
def sample(prediction):
    r = random.uniform(0, 1)
    s = 0 # stores prediction character
    char_id = len(prediction) - 1
    
    #for each char prediction probability
    for i in range(len(prediction)):
        s += prediction[i]
        if s >= r:
            char_id = i
            break
    
    #make one hot encoding
    char_one_hot = np.zeros(shape(char_size))
    char_one_hot[char_id] = 1.0
    
    return char_one_hot

In [10]:
#vectorize the data to feed it into the model
len_per_section = 50
skip = 2
sections = []
next_chars = []

for i in range(0, len(text) - len_per_section, skip):
    sections.append(text[i : i + len_per_section])
    next_chars.append(text[i + len_per_section])
    
#vectorize inputs
X = np.zeros((len(sections), len_per_section, char_size))
#label column for all the character id's, still zero
y = np.zeros((len(sections), char_size))
#for each char in each section, convert each char to an ID
#for each section convert the labels to ids 
for i, section in enumerate(sections):
    for j, char in enumerate(section):
        X[i, j, char2id[char]] = 1
    y[i, char2id[next_chars[i]]] = 1
print(y)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]]


In [12]:
#machine learning part

batch_size = 512
max_steps = 1000
log_every = 100
save_every = 6000
hidden_nodes = 1024 #https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw/1097#1097
test_start = 'I am thinking that'

#save the model
checkpoint_directory = 'ckpt'

#create a checkpoint directory
if tf.gfile.Exists(checkpoint_directory):
    tf.gfile.DeleteRecursively(checkpoint_directory)
tf.gfile.MakeDirs(checkpoint_directory)

print('training data size:', len(X))
print('approximate steps per epoch: ', int(len(X)/batch_size))

training data size: 644253
approximate steps per epoch:  1258


In [21]:
#build our model time
#create computation graph
graph = tf.Graph()
#if multiple graphs, but none here jsut one
with graph.as_default():
    ###########
    #Prep
    ###########
    #Variables and placeholders
    #global_step refer to the number of batches seen by the graph. 
    #Everytime a batch is provided, the weights are updated in the 
    #direction that minimizes the loss. global_step just keeps track 
    #of the number of batches seen so far starts off as 0
    global_step = tf.Variable(0)
    
    #data tensor shape feeding in sections
    data = tf.placeholder(tf.float32, [batch_size, len_per_section, char_size])
    #labels
    labels = tf.placeholder(tf.float32, [batch_size, char_size])
    
    #An LSTM RNN (Long Short Term Memory), consists of 3 gates and an internal state, 
    #This enables the LSTM to capture long-term dependencies. 
    #http://suriyadeepan.github.io/2017-02-13-unfolding-rnn-2/
    #lets build weights and biases for each of the 3 gates and then for the cell state
    
    #tf variables
    #Since we need the weights and biases for our model. 
    #We could imagine treating these like additional inputs, 
    #but TensorFlow has an even better way to handle it: Variable
    #A Variable is a modifiable tensor that lives in TensorFlow's graph of 
    #interacting operations. It can be used and even modified by the computation. 
    #For machine learning applications, one generally has the model parameters be Variables.
    
    #Prep LSTM Operation
    #Input gate: weights for input, weights for previous output, and bias
    
    #tf truncated normal
    #Outputs random values from a truncated normal distribution.
    #The generated values follow a normal distribution with specified mean and 
    #standard deviation, except that values whose magnitude is more than 2 standard deviations
    #from the mean are dropped and re-picked.
    #basically randomly initialized values here
    
    #biases act as an anchor

    w_ii = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_io = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_i = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Forget gate: weights for input, weights for previous output, and bias
    w_fi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_fo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_f = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Output gate: weights for input, weights for previous output, and bias
    w_oi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_oo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_o = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Memory cell: weights for input, weights for previous output, and bias
    w_ci = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_co = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_c = tf.Variable(tf.zeros([1, hidden_nodes]))
    
    
    #LSTM Cell
    # given input, output, external state, it will return output and state
    #output starts off empty, LSTM cell calculates it
    
    #Since, we have two kinds of states - the internal state ct 
    #and the (exposed) external state st, and since we need both of 
    #them for the subsequent sequential operations, we combine them 
    #into a tensor at each step, and pass them as input to the next 
    #step. This tensor is unpacked into st_1 and ct_1 at the beginning of each step.
    
    
    def lstm(i, o, state):
        
        #these are all calculated seperately, no overlap until....
        #(input * input weights) + (output * weights for previous output) + bias
        input_gate = tf.sigmoid(tf.matmul(i, w_ii) + tf.matmul(o, w_io) + b_i)
        #(input * forget weights) + (output * weights for previous output) + bias
        forget_gate = tf.sigmoid(tf.matmul(i, w_fi) + tf.matmul(o, w_fo) + b_f)
        #(input * output weights) + (output * weights for previous output) + bias
        output_gate = tf.sigmoid(tf.matmul(i, w_oi) + tf.matmul(o, w_oo) + b_o)
        #(input * internal state weights) + (output * weights for previous output) + bias
        memory_cell = tf.sigmoid(tf.matmul(i, w_ci) + tf.matmul(o, w_co) + b_c)
        
        #...now! multiply forget gate * given state    +  input gate * hidden state
        state = forget_gate * state + input_gate * memory_cell
        #squash that state with tanh nonlin (Computes hyperbolic tangent of x element-wise)
        #multiply by output
        output = output_gate * tf.tanh(state)
        #return 
        return output, state
    
    ###########
    #Operation
    ###########
    #LSTM
    #both start off as empty, LSTM will calculate this
    output = tf.zeros([batch_size, hidden_nodes])
    state = tf.zeros([batch_size, hidden_nodes])

    #unrolled LSTM loop
    #for each input set
    for i in range(len_per_section):
        #calculate state and output from LSTM
        output, state = lstm(data[:, i, :], output, state)
        #to start, 
        if i == 0:
            #store initial output and labels
            outputs_all_i = output
            labels_all_i = data[:, i+1, :]
        #for each new set, concat outputs and labels
        elif i != len_per_section - 1:
            #concatenates (combines) vectors along a dimension axis, not multiply
            outputs_all_i = tf.concat([outputs_all_i, output], 0)
            labels_all_i = tf.concat([labels_all_i, data[:, i+1, :]], 0)
        else:
            #final store
            outputs_all_i = tf.concat([outputs_all_i, output], 0)
            labels_all_i = tf.concat([labels_all_i, labels], 0)
        
    #Classifier
    #The Classifier will only run after saved_output and saved_state were assigned.
    
    #calculate weight and bias values for the network
    #generated randomly given a size and distribution
    w = tf.Variable(tf.truncated_normal([hidden_nodes, char_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([char_size]))
    #Logits simply means that the function operates on the unscaled output 
    #of earlier layers and that the relative scale to understand the units 
    #is linear. It means, in particular, the sum of the inputs may not equal 1, 
    #that the values are not probabilities (you might have an input of 5).
    logits = tf.matmul(outputs_all_i, w) + b
    
    #logits is our prediction outputs, lets compare it with our labels
    #cross entropy since multiclass classification
    #computes the cost for a softmax layer
    #then Computes the mean of elements across dimensions of a tensor.
    #average loss across all values
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_all_i))

    #Optimizer
    #minimize loss with graident descent, learning rate 10,  keep track of batches
    optimizer = tf.train.GradientDescentOptimizer(10.).minimize(loss, global_step=global_step)
    
    ###########
    #Test
    ###########
    #test_data = tf.placeholder(tf.float32, shape=[1, char_size])
    #test_output = tf.Variable(tf.zeros([1, hidden_nodes]))
    #test_state = tf.Variable(tf.zeros([1, hidden_nodes]))
    
    #Reset at the beginning of each test
    #reset_test_state = tf.group(test_output.assign(tf.zeros([1, hidden_nodes])), 
                                #test_state.assign(tf.zeros([1, hidden_nodes])))

    #LSTM
    #test_output, test_state = lstm(test_data, test_output, test_state)
    #test_prediction = tf.nn.softmax(tf.matmul(test_output, w) + b)

In [22]:
#timew to train the model, initialize a session with a graph
with tf.Session(graph=graph) as sess:
    #standard init step
    tf.global_variables_initializer().run()
    offset = 0
    saver = tf.train.Saver()
    
    #for each training step
    for step in range(max_steps):
        
        #starts off as 0
        offset = offset % len(X)
        
        #calculate batch data and labels to feed model iteratively
        if offset <= (len(X) - batch_size):
            #first part
            batch_data = X[offset: offset + batch_size]
            batch_labels = y[offset: offset + batch_size]
            offset += batch_size
        #until when offset  = batch size, then we 
        else:
            #last part
            to_add = batch_size - (len(X) - offset)
            batch_data = np.concatenate((X[offset: len(X)], X[0: to_add]))
            batch_labels = np.concatenate((y[offset: len(X)], y[0: to_add]))
            offset = to_add
        
        #optimize!!
        _, training_loss = sess.run([optimizer, loss], feed_dict={data: batch_data, labels: batch_labels})
        
        if step % 10 == 0:
            print('training loss at step %d: %.2f (%s)' % (step, training_loss, datetime.datetime.now()))

            if step % save_every == 0:
                saver.save(sess, checkpoint_directory + '/model', global_step=step)

training loss at step 0: 5.55 (2017-03-22 20:29:06.843898)
training loss at step 10: 4.81 (2017-03-22 20:31:19.035997)
training loss at step 20: 4.77 (2017-03-22 20:33:27.438215)
training loss at step 30: 3.84 (2017-03-22 20:35:35.079378)
training loss at step 40: 4.42 (2017-03-22 20:37:41.031519)
training loss at step 50: 4.17 (2017-03-22 20:39:45.641800)
training loss at step 60: 3.51 (2017-03-22 20:41:51.682201)


KeyboardInterrupt: 